## Configurando o MySQL com Python

Neste momento, vamos configurar a integração entre o MySQL e Python. Isso nos permitirá executar consultas, inserções e outras operações no banco de dados MySQL usando o Python como nossa linguagem de programação principal.

In [2]:
import os
from dotenv import load_dotenv
import mysql.connector

# Carrega as variáveis do arquivo .env no ambiente de trabalho
load_dotenv()

# A função os.getenv é usada para obter o valor das variáveis de ambiente
host = os.getenv("DB_HOST")
user = os.getenv("DB_USERNAME")
password = os.getenv("DB_PASSWORD")

cnx = mysql.connector.connect(
    host=host,
    user=user,
    password=password
)

print(cnx)

Criaremos um cursor, que é um objeto utilizado para executar instruções SQL no contexto do Python. Essse cursor nos permitirá enviar consultas e comandos SQL para o banco de dados MySQL por meio da conexão estabelecida e obter os resultados de volta para serem processados em nosso código Python

In [3]:
cursor = cnx.cursor()


## Criando uma base de dados

O método `execute()` é usado para compilar uma instrução SQL

In [4]:
cursor.execute("CREATE DATABASE IF NOT EXISTS dbprodutos;")

In [5]:
cursor.execute('SHOW DATABASES;')
for db in cursor:
    print(db)

('dbprodutos',)
('information_schema',)
('mysql',)
('performance_schema',)
('sys',)


## Criando uma tabela

Antes de criar nossa tabela, vamos revisar as colunas necessárias que precisaremos incluir nela. Essa etapa é importante para garantir que nossa tabela seja projetada corretamente, atendendo aos requisitos de armazenamento e organização dos dados.

In [6]:
import pandas as pd 

df_livros = pd.read_csv('/home/jarbas/Documentos/pipeline-python-mongo-mysql/data/tabela_livros.csv')
df_livros.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,latitude,longitude
0,67ce218856351bd2124a7c78,Modelagem preditiva,livros,92.45,5.609697,2020-01-01,Thiago Silva,BA,1,cartao_credito,3,-13.29,-41.71
1,67ce218856351bd2124a7c79,Iniciando em programação,livros,43.84,0.000000,2020-01-01,Mariana Ferreira,SP,5,cartao_credito,1,-22.19,-48.79
2,67ce218856351bd2124a7c8b,Iniciando em programação,livros,63.25,3.894137,2022-01-01,Juliana Costa,RJ,5,cartao_credito,4,-22.25,-42.66
3,67ce218856351bd2124a7c99,Ciência de dados com python,livros,86.13,5.273176,2021-02-01,Camila Ribeiro,RJ,4,cartao_credito,3,-22.25,-42.66
4,67ce218856351bd2124a7c9b,Ciência de dados com python,livros,72.75,1.458158,2021-02-01,Beatriz Moraes,PR,4,cartao_credito,2,-24.89,-51.55


In [7]:
df_livros.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'latitude', 'longitude'],
      dtype='object')

In [8]:
df_livros.shape

(742, 13)

In [9]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_livros(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               
               PRIMARY KEY (id));
""")

**Selecionando a base de dados para verificar a tabela criada**

In [10]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
  print(tb)

('tb_livros',)
('tb_produtos',)


## Inserindo os dados do csv na tabela

Para inserir os dados na tabela do MySQL, é necessário percorrer cada linha do DataFrame e transformá-las em tuplas. Essa abordagem permite que mapeemos os dados do DataFrame para as colunas correspondentes da tabela do MySQL de forma eficiente e precisa.

In [11]:
for i, row in df_livros.iterrows():
    print(tuple(row))

('67ce218856351bd2124a7c78', 'Modelagem preditiva', 'livros', 92.45, 5.6096965236, '2020-01-01', 'Thiago Silva', 'BA', 1, 'cartao_credito', 3, -13.29, -41.71)
('67ce218856351bd2124a7c79', 'Iniciando em programação', 'livros', 43.84, 0.0, '2020-01-01', 'Mariana Ferreira', 'SP', 5, 'cartao_credito', 1, -22.19, -48.79)
('67ce218856351bd2124a7c8b', 'Iniciando em programação', 'livros', 63.25, 3.894137208, '2022-01-01', 'Juliana Costa', 'RJ', 5, 'cartao_credito', 4, -22.25, -42.66)
('67ce218856351bd2124a7c99', 'Ciência de dados com python', 'livros', 86.13, 5.2731755368, '2021-02-01', 'Camila Ribeiro', 'RJ', 4, 'cartao_credito', 3, -22.25, -42.66)
('67ce218856351bd2124a7c9b', 'Ciência de dados com python', 'livros', 72.75, 1.458157705, '2021-02-01', 'Beatriz Moraes', 'PR', 4, 'cartao_credito', 2, -24.89, -51.55)
('67ce218856351bd2124a7c9e', 'Iniciando em programação', 'livros', 61.73, 0.0, '2021-02-01', 'Beatriz Moraes', 'SP', 4, 'cartao_credito', 1, -22.19, -48.79)
('67ce218856351bd2124a7c

In [12]:
lista_dados = [tuple(row) for i, row in df_livros.iterrows()]
lista_dados

[('67ce218856351bd2124a7c78',
  'Modelagem preditiva',
  'livros',
  92.45,
  5.6096965236,
  '2020-01-01',
  'Thiago Silva',
  'BA',
  1,
  'cartao_credito',
  3,
  -13.29,
  -41.71),
 ('67ce218856351bd2124a7c79',
  'Iniciando em programação',
  'livros',
  43.84,
  0.0,
  '2020-01-01',
  'Mariana Ferreira',
  'SP',
  5,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('67ce218856351bd2124a7c8b',
  'Iniciando em programação',
  'livros',
  63.25,
  3.894137208,
  '2022-01-01',
  'Juliana Costa',
  'RJ',
  5,
  'cartao_credito',
  4,
  -22.25,
  -42.66),
 ('67ce218856351bd2124a7c99',
  'Ciência de dados com python',
  'livros',
  86.13,
  5.2731755368,
  '2021-02-01',
  'Camila Ribeiro',
  'RJ',
  4,
  'cartao_credito',
  3,
  -22.25,
  -42.66),
 ('67ce218856351bd2124a7c9b',
  'Ciência de dados com python',
  'livros',
  72.75,
  1.458157705,
  '2021-02-01',
  'Beatriz Moraes',
  'PR',
  4,
  'cartao_credito',
  2,
  -24.89,
  -51.55),
 ('67ce218856351bd2124a7c9e',
  'Iniciando em progr

In [13]:
sql = "INSERT INTO dbprodutos.tb_livros VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(sql, lista_dados)
cnx.commit()

IntegrityError: 1062 (23000): Duplicate entry '67ce218856351bd2124a7c78' for key 'tb_livros.PRIMARY'

In [ ]:
print (cursor.rowcount, "registro(s) inserido(s).")

6574 registro(s) inserido(s).


In [ ]:
df_produtos = pd.read_csv('/home/jarbas/Documentos/pipeline-python-mongo-mysql/data/tabela_produtos.csv')
df_produtos.head()

,_id,Produto,Categoria do Produto,Preço,Frete,Data da Compra,Vendedor,Local da compra,Avaliação da compra,Tipo de pagamento,Quantidade de parcelas,latitude,longitude
0,67ce218856351bd2124a7c81,Xadrez de madeira,brinquedos,25.23,0.000000,2021-01-01,Thiago Silva,BA,5,cartao_credito,2,-13.29,-41.71
1,67ce218856351bd2124a7c82,Impressora,eletronicos,322.04,14.732100,2021-01-01,João Souza,SP,3,cartao_credito,1,-22.19,-48.79
2,67ce218856351bd2124a7c83,Mesa de centro,moveis,282.22,12.611805,2021-01-01,João Souza,SP,3,boleto,1,-22.19,-48.79
3,67ce218856351bd2124a7c84,Tablet ABXY,eletronicos,1100.42,68.451348,2021-01-01,Beatriz Moraes,SP,1,cupom,1,-22.19,-48.79
4,67ce218856351bd2124a7c85,Fogão,eletrodomesticos,791.81,42.444626,2021-01-01,Juliana Costa,SP,4,boleto,1,-22.19,-48.79


In [ ]:
df_produtos.columns

Index(['_id', 'Produto', 'Categoria do Produto', 'Preço', 'Frete',
       'Data da Compra', 'Vendedor', 'Local da compra', 'Avaliação da compra',
       'Tipo de pagamento', 'Quantidade de parcelas', 'latitude', 'longitude'],
      dtype='object')

In [ ]:
df_produtos.shape

(6574, 13)

In [ ]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS dbprodutos.tb_produtos(
               id VARCHAR(100),
               Produto VARCHAR(100),
               Categoria_Produto VARCHAR(100),
               Preco FLOAT(10,2),
               Frete FLOAT(10,2),
               Data_Compra DATE,
               Vendedor VARCHAR(100),
               Local_Compra VARCHAR(100),
               Avaliacao_Compra INT,
               Tipo_Pagamento VARCHAR(100),
               Qntd_Parcelas INT,
               Latitude FLOAT(10,2),
               Longitude FLOAT(10,2),
               
               PRIMARY KEY (id));
""")

In [ ]:
cursor.execute("USE dbprodutos;")
cursor.execute("SHOW TABLES;")

for tb in cursor:
  print(tb)

('tb_livros',)
('tb_produtos',)


In [ ]:
lista_dados_produtos = [tuple(row) for i, row in df_produtos.iterrows()]
lista_dados_produtos

[('67ce218856351bd2124a7c81',
  'Xadrez de madeira',
  'brinquedos',
  25.23,
  0.0,
  '2021-01-01',
  'Thiago Silva',
  'BA',
  5,
  'cartao_credito',
  2,
  -13.29,
  -41.71),
 ('67ce218856351bd2124a7c82',
  'Impressora',
  'eletronicos',
  322.04,
  14.7321002401,
  '2021-01-01',
  'João Souza',
  'SP',
  3,
  'cartao_credito',
  1,
  -22.19,
  -48.79),
 ('67ce218856351bd2124a7c83',
  'Mesa de centro',
  'moveis',
  282.22,
  12.6118050347,
  '2021-01-01',
  'João Souza',
  'SP',
  3,
  'boleto',
  1,
  -22.19,
  -48.79),
 ('67ce218856351bd2124a7c84',
  'Tablet ABXY',
  'eletronicos',
  1100.42,
  68.4513476183,
  '2021-01-01',
  'Beatriz Moraes',
  'SP',
  1,
  'cupom',
  1,
  -22.19,
  -48.79),
 ('67ce218856351bd2124a7c85',
  'Fogão',
  'eletrodomesticos',
  791.81,
  42.4446263246,
  '2021-01-01',
  'Juliana Costa',
  'SP',
  4,
  'boleto',
  1,
  -22.19,
  -48.79),
 ('67ce218856351bd2124a7c86',
  'Violão',
  'instrumentos musicais',
  316.84,
  17.1595536296,
  '2022-01-01',
  '

In [ ]:
sql = "INSERT INTO dbprodutos.tb_produtos VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
cursor.executemany(sql, lista_dados_produtos)
cnx.commit()

In [ ]:
print (cursor.rowcount, "registro(s) inserido(s).")

6574 registro(s) inserido(s).


## Visualizando os dados inseridos

In [ ]:
cursor.close()

True

In [ ]:
cnx.close()